# Prediction Accuracy Graph

In [1]:
import altair as alt
import altair_data_server
from altair_saver import save
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

%matplotlib inline

sns.set(style='darkgrid', context='notebook')
%config InlineBackend.figure_format = 'retina'

alt.data_transformers.enable('data_server')

DataTransformerRegistry.enable('data_server')

In [2]:
from src.config import FIGURES, FINAL_DATA, MODELS
import joblib

In [3]:
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

In [4]:
al_data = pd.read_csv(FINAL_DATA/'al_data_final.csv').sample(frac = 1)
abs_data = pd.read_csv(FINAL_DATA/'abs_data_final.csv').sample(frac = 1)

In [5]:
from sklearn.linear_model import RidgeCV

In [6]:
al_model = RidgeCV()
abs_model = RidgeCV()

In [7]:
X_al = al_data.drop('tc_act', axis = 1)
y_al = al_data['tc_act']

X_abs = abs_data.drop('tc_act', axis = 1)
y_abs = abs_data['tc_act']

In [8]:
al_scaler = StandardScaler()
abs_scaler = StandardScaler()

In [9]:
X_al_scaled = al_scaler.fit_transform(X_al)
X_abs_scaled = abs_scaler.fit_transform(X_abs)

In [10]:
al_model.fit(X_al_scaled, y_al)
abs_model.fit(X_abs_scaled, y_abs)

RidgeCV(alphas=array([ 0.1,  1. , 10. ]))

In [11]:
al_pred = al_model.predict(X_al_scaled)
abs_pred = abs_model.predict(X_abs_scaled)

In [12]:
al_data

,x,temp,nf_hz,amp_mm,tc_act
24,25,22,40.970000,12.010000,0.00
9,5,22,28.647826,15.118261,2.00
171,25,200,36.810000,13.240000,0.52
140,25,150,31.343333,17.402800,1.75
39,5,50,37.050000,11.440000,0.52
...,...,...,...,...,...
135,25,150,37.530000,11.560000,0.52
53,15,50,36.338904,13.558356,1.00
153,5,200,25.588462,19.240769,2.00
43,5,50,33.197455,15.486809,1.50


In [13]:
al_data['tc_pred'] = al_pred

In [14]:
al_data

,x,temp,nf_hz,amp_mm,tc_act,tc_pred
24,25,22,40.970000,12.010000,0.00,0.483516
9,5,22,28.647826,15.118261,2.00,1.963573
171,25,200,36.810000,13.240000,0.52,0.409357
140,25,150,31.343333,17.402800,1.75,1.619568
39,5,50,37.050000,11.440000,0.52,0.523971
...,...,...,...,...,...,...
135,25,150,37.530000,11.560000,0.52,0.370156
53,15,50,36.338904,13.558356,1.00,0.930402
153,5,200,25.588462,19.240769,2.00,2.003987
43,5,50,33.197455,15.486809,1.50,1.336988


In [15]:
abs_data['tc_pred'] = abs_pred

In [16]:
abs_data

,x,temp,nf_hz,amp_mm,tc_act,tc_pred
87,15,50,16.76,3.70,2.40,2.237817
82,15,50,25.77,5.21,1.30,1.263855
146,25,70,15.84,3.79,2.25,2.236172
123,25,60,18.36,4.25,2.10,2.015513
130,15,70,26.19,5.26,1.30,1.104845
...,...,...,...,...,...,...
104,5,60,19.96,4.68,2.00,1.690358
125,25,60,13.61,3.89,2.30,2.448725
15,15,23,24.84,4.64,1.90,1.599041
19,15,23,19.82,3.40,2.50,2.215068


In [17]:
fig1 = alt.Chart(al_data).mark_point().encode(
    x = alt.X('tc_act:Q', title = 'Actual Crack Depth (mm)'),
    y = alt.Y('tc_pred:Q', title = 'Predicted Crack Depth (mm)'),
    color = alt.Color('temp:O', title = 'Temperature (C)', scale = alt.Scale(scheme = 'reds'))
).properties(
    width = 800,
    height = 400,
    title = "Model Predictive Accuracy (Aluminium)"
).configure_axis(labelFontSize = 14).configure_legend(labelFontSize = 14)

save(fig1, fp = str(FIGURES/'aluminium_accuracy_altair.png'), fmt = 'png', method = 'selenium', scale_factor = 4.0)

In [18]:
fig1

alt.Chart(...)

In [19]:
fig2 = alt.Chart(abs_data).mark_point().encode(
    x = alt.X('tc_act:Q', title = 'Actual Crack Depth (mm)'),
    y = alt.Y('tc_pred:Q', title = 'Predicted Crack Depth (mm)'),
    color = alt.Color('temp:O', title = 'Temperature (C)', scale = alt.Scale(scheme = 'reds'))
).properties(
    width = 800,
    height = 400,
    title = "Model Predictive Accuracy (ABS)"
).configure_axis(labelFontSize = 14).configure_legend(labelFontSize = 14)

save(fig2, fp = str(FIGURES/'abs_accuracy_altair.png'), fmt = 'png', method = 'selenium', scale_factor = 4.0)

In [20]:
fig2

alt.Chart(...)

In [21]:
al_data['material'] = 'Aluminium'
abs_data['material'] = 'ABS'

In [22]:
combined = pd.concat([al_data, abs_data], ignore_index = True)

In [23]:
combined

,x,temp,nf_hz,amp_mm,tc_act,tc_pred,material
0,25,22,40.970000,12.010000,0.00,0.483516,Aluminium
1,5,22,28.647826,15.118261,2.00,1.963573,Aluminium
2,25,200,36.810000,13.240000,0.52,0.409357,Aluminium
3,25,150,31.343333,17.402800,1.75,1.619568,Aluminium
4,5,50,37.050000,11.440000,0.52,0.523971,Aluminium
...,...,...,...,...,...,...,...
324,5,60,19.960000,4.680000,2.00,1.690358,ABS
325,25,60,13.610000,3.890000,2.30,2.448725,ABS
326,15,23,24.840000,4.640000,1.90,1.599041,ABS
327,15,23,19.820000,3.400000,2.50,2.215068,ABS


In [24]:
# Clean up difference between two datasets
combined['temp'].replace({23: 22}, inplace = True)

In [25]:
fig3 = alt.Chart(combined).mark_circle(opacity = 0.5).encode(
    x = alt.X('tc_act:Q', title = 'Actual Crack Depth (mm)'),
    y = alt.Y('tc_pred:Q', title = 'Predicted Crack Depth (mm)'),
    color = alt.Color('material:N', title = 'Material'),
    size = alt.Size('temp:Q', title = 'Temperature (C)')
).properties(
    width = 800,
    height = 400,
    title = "Model Predictive Accuracy"
).configure_axis(labelFontSize = 14).configure_legend(labelFontSize = 14)

save(fig3, fp = str(FIGURES/'combined_accuracy_altair.png'), fmt = 'png', method = 'selenium', scale_factor = 4.0)

In [26]:
fig3

alt.Chart(...)

In [27]:
# Because I can't decide which looks better, let's have both!

In [28]:
fig3b = alt.Chart(combined).mark_circle(opacity = 0.5).encode(
    x = alt.X('tc_act:Q', title = 'Actual Crack Depth (mm)'),
    y = alt.Y('tc_pred:Q', title = 'Predicted Crack Depth (mm)'),
    color = alt.Color('material:N', title = 'Material'),
    size = alt.Size('temp:O', title = 'Temperature (C)')
).properties(
    width = 800,
    height = 400,
    title = "Model Predictive Accuracy"
).configure_axis(labelFontSize = 14).configure_legend(labelFontSize = 14)

save(fig3b, fp = str(FIGURES/'combined_accuracy_altair_ordinal.png'), fmt = 'png', method = 'selenium', scale_factor = 4.0)

In [29]:
fig3b

alt.Chart(...)